In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

path_to_folder = "/content/drive/MyDrive/Data/Covid19 data/FinalDataSet"

cough_files = os.listdir(path_to_folder)





In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import os
import numpy as np

In [ ]:
for file in os.listdir('/content/drive/MyDrive/Data/Covid19 data/FinalDataSet'):
  print(file)

005b8518-03ba-4bf5-86d2-005541442357(healthy dry).wav
038592cb-c8db-4f55-8052-e20059146cb5(healthy dry).wav
040e6a1c-dcbe-47aa-8b9d-51abd8136d91(healthy dry).wav
06b568b5-b9f8-4334-816c-c16009bb5de7(covid wet).wav
0733f882-d7fd-4dc5-a1b0-8aeec64fc112(healthy dry).wav
0762cd0a-5ce6-4b1b-9374-5d4396c218ed(covid unknown).wav
075cdb5a-5c29-4266-84ea-c44963bd0454(healthy dry).wav
07e33d6b-560d-4c16-8d7e-ecd5ddb3b195(healthy dry).wav
07e2f7e1-8e9b-4962-8701-fbe1224de7ef(healthy dry).wav
082991fb-ee66-4970-91dd-50890f4cc9b1(covid dry).wav
08e1fcba-d1b0-4365-b1ac-5efa5d143bcb(healthy unknown).wav
0924c308-2cca-4ab2-8ef3-a204c349701b(covid unknown).wav
09bffe1a-63f0-4f2e-942b-f7a152b344fc(healthy dry).wav
09ef22ac-58b8-4f3b-b00a-94f926298dae(healthy dry).wav
1122a43f-1a4b-4292-903e-963b8463a56c(healthy dry).wav
126b940c-b6c6-4d3d-adc2-92aafab911b4(covid unknown).wav
128aac08-4a43-4a36-ac34-5f294d90ca2e(covid unknown).wav
1351a8e1-f5e5-4a05-a6a9-4bac18393022(healthy dry) (1).wav
1351a8e1-f5e5-4a

In [ ]:
import os
import librosa
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define function to extract audio features
def extract_features(audio_path):
    # Load audio file
    signal, sr = librosa.load(audio_path, sr=None)
    # Extract features Mel-frequency cepstral coefficients
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=40)

    # Return features
    return {'mfccs': mfccs}

# Set directory path
directory = '/content/drive/MyDrive/Data/Covid19 data/FinalDataSet'

# Initialize empty dictionary to store features
features = {}

# Loop through all files in directory
for audio in os.listdir(directory):
    # Check if file is an audio file
    if audio.endswith('.wav'):
        # Construct path to audio file
        audio_path = os.path.join(directory, audio)

        # Extract features from audio file
        audio_features = extract_features(audio_path)

        # Store features in dictionary with file name as key
        features[audio] = audio_features

# Save features dictionary to a numpy file
np.save('/content/drive/MyDrive/Data/Covid19 data/features.npy', features)


In [ ]:
from google.colab import files

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Define function to extract audio features
def extract_features(audio_path):
    # Load audio file
    signal, sr = librosa.load(audio_path, sr=None)
    # Extract features Mel-frequency cepstral coefficients
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=40)

    # Return features
    return {'mfccs': mfccs}

# Load features and labels
features = np.load('/content/drive/MyDrive/Data/Covid19 data/features.npy', allow_pickle=True).item()

# Define a function to extract the label from the filename
def extract_label(filename):
    if 'covid' in filename and 'dry' in filename:
        return 'covid dry'
    elif 'covid' in filename and 'cough' in filename:
        return 'covid cough'
    elif 'healthy' in filename and 'dry' in filename:
        return 'healthy dry'
    elif 'healthy' in filename and 'cough' in filename:
        return 'healthy cough'
    elif 'healthy' in filename and 'unknown' in filename:
        return 'healthy unknown'
    else:
        return 'covid unknown'

# Create feature and label arrays
X = []
y = []
for file, feature in features.items():
    X.append((feature['mfccs'].mean(axis=1)))
    label = extract_label(file)
    if label:
        y.append(label)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Gradient Boosting Machine classifier
clf = GradientBoostingClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Prompt the user to enter the audio file path
#file_path = input("Enter the path of the .wav audio file: ")

# Extract features from the user input file
#user_features = extract_features(file_path)


# Upload the file
uploaded = files.upload()

# Get the file name
file_name = list(uploaded.keys())[0]
file_path = '/content/' + file_name

user_features = extract_features(file_path)

# Convert the features to a 2D array
user_X = np.array(user_features['mfccs'].mean(axis=1)).reshape(1, -1)

# Predict the cough label for the user input
user_pred = clf.predict(user_X)

# Print the predicted label
print(f'Predicted Cough Label: {user_pred[0]}')


Saving aaa.wav to aaa.wav


<ipython-input-50-4c6e921a4a25>:9: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(audio_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Predicted Cough Label: healthy dry


In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

#from django.utils.functional import safestring

cred = credentials.Certificate("/content/drive/MyDrive/Data/cse499-cd2ac-firebase-adminsdk-6gvyp-a7b92ab7dd.json")
firebase_admin.initialize_app(cred,{'databaseURL': 'https://cse499-cd2ac-default-rtdb.firebaseio.com'})

In [ ]:
root = db.reference()
cough1=user_pred[0]
print(cough1)
root.child('Data').child('Cough').set(cough1)
dt = root.child('Data').get()
print(dt)

healthy dry
{'BPM': 31, 'Cough': 'healthy dry', 'SP02': 95, 'Temperature': 33.9375}


In [ ]:
from joblib import dump

In [ ]:
dump(clf, '/content/sample_data/cough.joblib')

['/content/sample_data/cough.joblib']